In [1]:
from google.colab import drive, files
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import mean_squared_error

In [3]:
df = pd.read_csv('/content/drive/MyDrive/NN/files/data/wine_data_final.csv')

We separated the thick-skin grapes from the thin-skin grapes for they have different growth patterns. This model below represents the thick grapes (Cabernet Sauvignon, Zinfandel, Syrah). Realized there was no snow at all throughout the season, so we got rid of that feature just for this model. Some years had some very corrupt weather data, so some years had to be discarded which may have affected our predictions. We conducted this using 5-fold cross-validation for the purpose of using all of the data for both training and testing. I tried multiple models such as Lasso, Ridge, and Linear Regression from the sklearn library, all of which were very similar. The RMSE for each model is around 2.4 which isn't too bad. With the lack of some weather features and only 233 ratings to base our model off of, 2.4 is actually pretty good. The range of the real ratings was between 84 and 96 with a mean of 91.

In [4]:
df_0 = df[df['skin'] == 0] # 96
df_avg_0 = pd.DataFrame(columns = ['tavg','tmin','tmax','prcp','snow','wspd','pres','quality'])
j = 0
for i, rows in df_0.iterrows():
  weather = pd.read_csv('/content/drive/MyDrive/NN/files/data/complete_weather/' + df_0['file'][i])
  df_avg_0 = df_avg_0.append(weather.mean(), ignore_index=True)
  df_avg_0['quality'][j] = df_0['quality'][i]
  j += 1
df_avg_0 = df_avg_0.dropna() # now 83
del df_avg_0['snow']
# display(df_avg_0)

In [5]:
x_0 = df_avg_0[['tavg','tmin','tmax','prcp','wspd','pres']].to_numpy() # features
y_0 = df_avg_0['quality'].to_numpy() # target

model = LinearRegression()
y_pred_0 = cross_val_predict(model, x_0, y_0, cv = 5)
# print(y_0)
# print(y_pred_0)
print("RMSE: %.3f" % np.sqrt(mean_squared_error(y_0, y_pred_0)))

RMSE: 2.336


This model below represents the thin grapes (Pinot Noir and Chardonnay)

In [6]:
df_1 = df[df['skin'] == 1] # 127
df_avg_1 = pd.DataFrame(columns = ['tavg','tmin','tmax','prcp','snow','wspd','pres','quality'])
j = 0
for i, rows in df_1.iterrows():
  weather = pd.read_csv('/content/drive/MyDrive/NN/files/data/complete_weather/' + df_1['file'][i])
  df_avg_1 = df_avg_1.append(weather.mean(), ignore_index=True)
  df_avg_1['quality'][j] = df_1['quality'][i]
  j += 1
df_avg_1 = df_avg_1.dropna() # now 115
del df_avg_1['snow']
# display(df_avg_1)

In [7]:
x_1 = df_avg_1[['tavg','tmin','tmax','prcp','wspd','pres']].to_numpy() # features
y_1 = df_avg_1['quality'].to_numpy() # target

model = LinearRegression()
y_pred_1 = cross_val_predict(model, x_1, y_1, cv = 5)
# print(y_1)
# print(y_pred_1)
print("RMSE: %.3f" % np.sqrt(mean_squared_error(y_1, y_pred_1)))

RMSE: 2.451


CNN

In [8]:
from keras.models import Sequential
import tensorflow as tf
from keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [28]:
x_0 = x_0.reshape(x_0.shape[0], x_0.shape[1], 1)

xtrain, xtest, ytrain, ytest = train_test_split(x_0, y_0, test_size=0.33)

model = Sequential()
model.add(Conv1D(100, 1, activation='relu', input_shape=(6,1)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])

model.fit(xtrain, ytrain, epochs=2000)

Epoch 1/2000
2/2 [==============================] - 1s 15ms/step - loss: 8143.4644 - root_mean_squared_error: 90.2411
Epoch 2/2000
2/2 [==============================] - 0s 14ms/step - loss: 7308.6763 - root_mean_squared_error: 85.4908
Epoch 3/2000
2/2 [==============================] - 0s 21ms/step - loss: 6638.9092 - root_mean_squared_error: 81.4795
Epoch 4/2000
2/2 [==============================] - 0s 12ms/step - loss: 6077.6421 - root_mean_squared_error: 77.9592
Epoch 5/2000
2/2 [==============================] - 0s 13ms/step - loss: 5566.2070 - root_mean_squared_error: 74.6070
Epoch 6/2000
2/2 [==============================] - 0s 15ms/step - loss: 5089.1875 - root_mean_squared_error: 71.3385
Epoch 7/2000
2/2 [==============================] - 0s 14ms/step - loss: 4609.9263 - root_mean_squared_error: 67.8964
Epoch 8/2000
2/2 [==============================] - 0s 15ms/step - loss: 4121.1299 - root_mean_squared_error: 64.1960
Epoch 9/2000
2/2 [==============================] - 0s 1